# Модуль 3. Мультимодальные и мультизадачные модели. Часть 1

Это вторая часть домашней работы №3 "Реализация Visual Question Answering / Document Question Answering"

## Часть 2. Использование модели

**Цель:** отработать навыки адаптации готовых моделей для решения прикладной задачи на русском языке, а также создание небольших демо для задач.

**В каком виде прислать результат:**

заполненный jupyter-notebook и видеозаписи работы с демо

### [2 балла] Добавить модель переводчика

У вас уже есть готовая модель, которая может по картинке отвечать на текстовые запросы к картинке. Ваша цель --- обобщить эту модель на русский язык, добавив модель переводчик, которая будет переводить запрос на русском языке в запрос на английском языке и передавать его модели. За основу вы можете взять языковую модель (например, https://huggingface.co/Helsinki-NLP/opus-mt-ru-en). Альтернативой может стать реализация функции, делающий api вызов, к приложению переводчика (например, https://libretranslate.com/).

---

**Ожидаемый результат**

В качестве результата в этой секции вам нужно предоставить функции, которые делают перевод с русского на английского и делает инференс модели DocVQA и выводит ответ на русском языке. (В качестве примеров вопросов, можете использовать данные из датасета)


In [1]:
!pip install -q transformers datasets
!pip install -q sentencepiece
!pip install -q git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
!pip install -q opencv-python
!pip install -q git+https://github.com/facebookresearch/detectron2.git
!pip install -q gradio
!pip install -q pillow
!pip install -q SpeechRecognition pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (se

In [1]:
from transformers import MarianMTModel
from transformers import MarianTokenizer
from transformers import LayoutLMv2Processor
from transformers import LayoutLMv2ForQuestionAnswering
from transformers import LayoutLMv2Tokenizer

import torch
import torchvision.transforms as transforms

import gradio as gr
import speech_recognition as sr

from PIL import Image

from huggingface_hub import notebook_login
notebook_login()

In [2]:
# Загрузка моделей и токенизаторов для обоих направлений перевода
models = {
    "ru-en": {
        "model_name": "Helsinki-NLP/opus-mt-ru-en",
        "model": None,
        "tokenizer": None
    },
    "en-ru": {
        "model_name": "Helsinki-NLP/opus-mt-en-ru",
        "model": None,
        "tokenizer": None
    }
}

# Функция инициализации модели и токенизатора (если они еще не были загружены)
def load_model_and_tokenizer(direction):
    if models[direction]["model"] is None or models[direction]["tokenizer"] is None:
        models[direction]["model"] = MarianMTModel.from_pretrained(models[direction]["model_name"])
        models[direction]["tokenizer"] = MarianTokenizer.from_pretrained(models[direction]["model_name"])

# Функция для перевода текста
def translate(text, direction):
    load_model_and_tokenizer(direction)
    tokenizer = models[direction]["tokenizer"]
    model = models[direction]["model"]
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

In [3]:
# Проверка функций перевода
text_ru = "Давайте поговорим об искусственном интеллекте!"
text_en = "Let's talk about artificial intelligence!"

translated_text_ru_en = translate(text_ru, "ru-en")
translated_text_en_ru = translate(text_en, "en-ru")

print("Оригинальный текст RU:", text_ru)
print("Переведенный текст RU-EN:", translated_text_ru_en)
print('-'*73)
print("Оригинальный текст EN:", text_en)
print("Переведенный текст EN-RU:", translated_text_en_ru)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Оригинальный текст RU: Давайте поговорим об искусственном интеллекте!
Переведенный текст RU-EN: Let's talk about artificial intelligence!
-------------------------------------------------------------------------
Оригинальный текст EN: Let's talk about artificial intelligence!
Переведенный текст EN-RU: Давайте поговорим об искусственном интеллекте!


In [4]:
# Загрузка модели, процессора и токенезатора
model_docvqa = LayoutLMv2ForQuestionAnswering.from_pretrained("NeKonnn/layoutlmv2-base-uncased_finetuned_docvqa")
processor_docvqa = LayoutLMv2Processor.from_pretrained("NeKonnn/layoutlmv2-base-uncased_finetuned_docvqa")
tokenizer_docvqa = LayoutLMv2Tokenizer.from_pretrained("NeKonnn/layoutlmv2-base-uncased_finetuned_docvqa")

In [5]:
def ru_inference(image_path, question_ru):
    '''
    Эта функция выполняет обработку изображения и вопроса на русском языке,
    использует модель для извлечения ответа на заданный вопрос из изображения,
    и возвращает ответ на русском языке.

    Параметры:
    image_path: путь к изображению, на котором нужно найти ответ.
    question_ru: вопрос на русском языке, на который нужно ответить.

    Возвращает:
    answer_ru: ответ на вопрос, извлеченный из изображения, на русском языке.
    '''


    image = Image.open(image_path).convert("RGB")
    question_en = translate(question_ru, "ru-en")
    inputs = processor_docvqa(image, question_en, return_tensors="pt")
    outputs = model_docvqa(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Игнорирование токенов CLS, SEP и PAD для начала и конца ответа
    ignore_index = [tokenizer_docvqa.cls_token_id, tokenizer_docvqa.sep_token_id, tokenizer_docvqa.pad_token_id]
    answer_start_scores[:, ignore_index] = -float("Inf")
    answer_end_scores[:, ignore_index] = -float("Inf")

    # Выбор начального и конечного токенов ответа
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Преобразование индексов в токены и очистка от служебных символов
    answer_tokens = tokenizer_docvqa.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
    answer_tokens = [token for token in answer_tokens if token not in (tokenizer_docvqa.cls_token, tokenizer_docvqa.sep_token, tokenizer_docvqa.pad_token)]

    # Сборка токенов в строку ответа
    answer = tokenizer_docvqa.convert_tokens_to_string(answer_tokens)
    answer_ru = translate(answer, "en-ru")

    return answer_ru

In [8]:
! pip install transformers datasets
! pip install 'git+https://github.com/facebookresearch/detectron2.git'
!sudo apt install tesseract-ocr
!pip install -q pytesseract

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-t_y_60o4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-t_y_60o4
  Resolved https://github.com/facebookresearch/detectron2.git to commit 337ca3490fa7879ceeeadf6c2b73d67504ff4b4f
  Preparing metadata (setup.py) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/univ

In [6]:
answer = ru_inference("/content/sample_data/text2.jpg", "Что такое информационные технологии?")
print("Ответ:", answer)

Ответ: Компьютерные системы или устройства для доступа к информации. Информационные технологии оказывают значительное влияние на нашу повседневную жизнь. Информационные технологии используются всеми предприятиями вплоть до одного человека и местных операций. Глобальные компании используют их для управления данными и модернизации своих процессов. Даже продавцы смартфонов используют для сбора платежей, а уличные исполнители дают имя для сбора пожертвований. Если вы используете таблицу для каталога, который вы покупаете Рождеством, вы используете информационную технологию.


### [2 балла] Сделать демо на gradio

Модель готова! Теперь было бы круто, если модель можно было захостить и оттестировать на практике. В этом задании вам нужно будет реализовать демо на gradio, которое будет принимать изображение и вопрос, а далее выдавать ответ. Пример демо, аналогично которому вам нужно реализовать модель --- https://huggingface.co/spaces/nielsr/comparing-VQA-models.


**Подсказка:**

В вкладке `Files` на демо вы можете посмотреть реализацию, там нужно заменить инференс, используемой модели, на инференс нашей модели с переводом


**Ожидаемый результат**

В качестве результата в этой секции вам нужно код для запуска демо на градио и видеозапись его работы, где реализован описанный выше функционал. Видео прикрепляйте отдельным файлом.

In [7]:
def ru_inference(image, question_ru):
    '''
    Функция для обработки изображения и вопроса на русском языке,
    для извлечения ответа на заданный вопрос с помощью модели документного вопросно-ответного анализа (DocVQA).
    Ответ извлекается из текста на изображении и переводится на русский язык.

    Параметры:
    image: изображение в формате PIL.Image, на котором нужно найти ответ.
    question_ru: вопрос на русском языке, на который нужно получить ответ из изображения.

    Возвращает:
    answer_ru: ответ на вопрос, извлечённый из текста на изображении и переведённый на русский язык.
    '''

    # Преобразование изображения в тензор PyTorch
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    image = transform(image).unsqueeze(0)


    question_en = translate(question_ru, "ru-en")
    inputs = processor_docvqa(image, question_en, return_tensors="pt")
    outputs = model_docvqa(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Игнорирование служебных токенов при выборе начального и конечного индексов
    ignore_index = [tokenizer_docvqa.cls_token_id, tokenizer_docvqa.sep_token_id, tokenizer_docvqa.pad_token_id]
    answer_start_scores[:, ignore_index] = -float("Inf")
    answer_end_scores[:, ignore_index] = -float("Inf")

    # Выбор начального и конечного токенов для ответа
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Извлечение токенов ответа
    answer_tokens = tokenizer_docvqa.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
    answer_tokens = [token for token in answer_tokens if token not in (tokenizer_docvqa.cls_token, tokenizer_docvqa.sep_token, tokenizer_docvqa.pad_token)]

    # Преобразование токенов в строку ответа
    answer = tokenizer_docvqa.convert_tokens_to_string(answer_tokens)
    answer_ru = translate(answer, "en-ru")

    return answer_ru

In [8]:
# Определение интерфейса Gradio
iface = gr.Interface(
    fn=ru_inference,
    inputs=[gr.Image(type="pil"), gr.Textbox(lines=2, placeholder="Введите вопрос на русском языке")],
    outputs=gr.Textbox(),
)

# Запуск Gradio интерфейса
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8844cf670b9e9a1915.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8844cf670b9e9a1915.gradio.live


### [4 балла] Ответы на вопросы голосом

Демо готово! Но кто хочет писать вопросы текстом?
Здесь вам предстоить улучшить ваше демо, чтобы оно могло принимать вопросы голосом. За основу вам предлагается рассмотреть демо https://www.gradio.app/guides/real-time-speech-recognition и добавить соответствуещее окошко в ваше демо. Также вы можете добавить text-to-speech модель, чтобы оно озвучило текстовый ответ (дополнительный балл к оценке)

---

**Ожидаемый результат**

В качестве результата в этой секции вам нужно код для запуска демо на градио и видеозапись его работы, где реализован описанный выше функционал.

In [9]:
!pip install gTTS

In [91]:
# Импортируем необходимые библиотеки для TTS
from gtts import gTTS
import tempfile
from io import BytesIO
import io
import os

In [20]:
import logging

In [81]:
# Функция преобразования текста в речь
def text_to_speech(text, lang='ru'):
    """
    Преобразование данного текста в аудио.

    Параметры:
    text (str): Текст для преобразования в речь.
    lang (str): Языковой код для TTS.

    Возвращает:
    BytesIO: Буфер байтов аудиофайла.
    """
    try:
        tts = gtts.gTTS(text=text, lang=lang)
        # Создаем временный файл для сохранения аудио
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
        tts.save(temp_file.name)
        return temp_file.name
    except Exception as e:
        logging.exception(f"An unexpected error occurred in text_to_speech: {e}")
        return None

In [82]:
# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Использование логирования в функции recognize_speech
def recognize_speech(audio_path):
    r = sr.Recognizer()
    try:
        with sr.AudioFile(audio_path) as source:
            audio = r.record(source)

        logging.info("Аудиоданные успешно загружены")
        text = r.recognize_google(audio, language='ru-RU')
        logging.info("Распознавание речи прошло успешно")
        return text
    except sr.UnknownValueError:
        logging.error("Сервис Google Speech Recognition не смог понять аудио")
    except sr.RequestError as e:
        logging.error(f"Не удалось отправить запрос в сервис Google Speech Recognition; {e}")
    except Exception as e:
        logging.exception(f"Произошла непредвиденная ошибка в recognize_speech: {e}")
    return None

ERROR:root:This is an error message


In [83]:
# Функция обработки изображения и вопроса
def ru_inference(image, audio=None):
    try:
        if audio is None:
            logging.error("В функцию не был предоставлен аудио вход.")
            return None

        # Распознавание речи из аудио
        question_ru = recognize_speech(audio)
        if question_ru is None:
            raise ValueError("Не удалось распознать вопрос из аудио.")

        # Перевод вопроса на английский
        question_en = translate(question_ru, "ru-en")

        # Преобразование изображения в тензор PyTorch
        transform = transforms.Compose([transforms.ToTensor()])
        image_tensor = transform(image).unsqueeze(0)

        # Препроцессинг изображения и вопроса и получение ответа моделью
        inputs = processor_docvqa(image_tensor, question_en, return_tensors="pt")
        outputs = model_docvqa(**inputs)

        # Игнорирование служебных токенов и извлечение ответа
        ignore_tokens = [tokenizer_docvqa.cls_token_id, tokenizer_docvqa.sep_token_id, tokenizer_docvqa.pad_token_id]
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits
        answer_start_scores[:, ignore_tokens] = -float("Inf")
        answer_end_scores[:, ignore_tokens] = -float("Inf")

        answer_start = torch.argmax(answer_start_scores, dim=1)
        answer_end = torch.argmax(answer_end_scores, dim=1) + 1

        # Извлечение токенов ответа и преобразование их в строку
        answer_tokens = tokenizer_docvqa.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
        answer_tokens = [token for token in answer_tokens if token not in (tokenizer_docvqa.cls_token, tokenizer_docvqa.sep_token, tokenizer_docvqa.pad_token)]
        answer = tokenizer_docvqa.convert_tokens_to_string(answer_tokens)

        # Перевод ответа на русский и преобразование в речь
        answer_ru = translate(answer, "en-ru")
        answer_audio_path = text_to_speech(answer_ru)

        return answer_audio_path

    except Exception as e:
        logging.exception(f"An error occurred in ru_inference: {e}")
        return None

In [101]:
import numpy as np
from scipy.io.wavfile import write

In [105]:
# Функция для записи и сохранения аудио
def save_audio(audio_data, filename='audio.wav', sample_rate=44100):
    if isinstance(audio_data, str):
        with open(audio_data, 'rb') as f:
            audio_data = f.read()
    elif isinstance(audio_data, io.BytesIO):
        audio_data = audio_data.read()

    audio_data = np.frombuffer(audio_data, dtype=np.int16)
    path = f'/content/sample_data{filename}'
    write(path, sample_rate, audio_data)
    return path

In [106]:
# Функция-обёртка для Gradio, которая обрабатывает аудио и изображение
def gradio_wrapper(image, audio_filepath):
    if audio_filepath is None:
        logging.error("No audio file path was provided.")
        return None
    logging.info(f"Audio file path received: {audio_filepath}")

    if not os.path.exists(audio_filepath):
        logging.error(f"Audio file does not exist at path: {audio_filepath}")
        return None

    new_audio_path = save_audio(audio_filepath, filename='new_audio.wav')
    logging.info(f"Audio file was saved to a new location: {new_audio_path}")

    answer_audio_path = ru_inference(image, new_audio_path)
    return answer_audio_path

In [107]:
# Создание интерфейса Gradio

iface = gr.Interface(
    fn=gradio_wrapper,
    inputs=[
        gr.Image(type="pil"),
        gr.Audio(sources="microphone", type="filepath", label="Запишите ваш вопрос")
    ],
    outputs=[
        gr.Audio(label="Озвученный ответ")
    ],
    title="Интерфейс для вопросов и ответов",
    description="Загрузите изображение и введите или скажите свой вопрос."
)

# Запуск интерфейса Gradio
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://548d01825218662c23.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ERROR:root:No audio file path was provided.
ERROR:root:Сервис Google Speech Recognition не смог понять аудио
ERROR:root:An error occurred in ru_inference: Не удалось распознать вопрос из аудио.
Traceback (most recent call last):
  File "<ipython-input-83-b5614723b5db>", line 11, in ru_inference
    raise ValueError("Не удалось распознать вопрос из аудио.")
ValueError: Не удалось распознать вопрос из аудио.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://548d01825218662c23.gradio.live
